In [ ]:
!pip install datasets transformers evaluate accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.0 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
import evaluate
import numpy as np


In [ ]:
dataset = load_dataset("imdb")
print(dataset)


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=256)

tokenized_dataset = dataset.map(preprocess_function, batched=True)
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


In [ ]:
small_train = tokenized_dataset["train"].shuffle(seed=42).select(range(25000))
small_test = tokenized_dataset["test"].shuffle(seed=42).select(range(25000))


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)


In [ ]:
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy.compute(predictions=predictions, references=labels)["accuracy"],
        "f1": f1.compute(predictions=predictions, references=labels, average="weighted")["f1"]
    }


In [ ]:
import sys
!{sys.executable} -m pip install -U transformers datasets evaluate accelerate huggingface_hub


In [ ]:
import transformers, datasets
print("transformers:", transformers.__version__)
print("datasets:", datasets.__version__)


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    fp16=True,  # ✅ Mixed precision (saves VRAM on your RTX 3050)
    logging_dir="./logs",
    logging_steps=50,
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train,
    eval_dataset=small_test,
    compute_metrics=compute_metrics,
)


In [ ]:
trainer.train()


In [ ]:
results = trainer.evaluate()
print(f"Accuracy: {results['eval_accuracy']:.4f}")
print(f"F1 Score: {results['eval_f1']:.4f}")


In [ ]:
import torch

# Make sure to move your model to device only once, not inside the function
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def predict_sentiment(text):
    # Tokenize and move inputs to the same device
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=256).to(device)

    # Disable gradient calculations for inference
    with torch.no_grad():
        outputs = model(**inputs)
        pred = outputs.logits.argmax(-1).item()

    return "Positive 😄" if pred == 1 else "Negative 😞"

In [ ]:

# Test
print(predict_sentiment("The movie was absolutely amazing, loved every part!"))
print(predict_sentiment("It was boring and way too long."))

In [ ]:
model.save_pretrained("./finetuned_imdb_model")
tokenizer.save_pretrained("./finetuned_imdb_model")


In [ ]:
import gradio as gr

def classify_review(review):
    inputs = tokenizer(review, return_tensors="pt", truncation=True, max_length=256)
    inputs = {k: v.to("cuda") for k, v in inputs.items()}
    outputs = model(**inputs)
    pred = outputs.logits.argmax(-1).item()
    return "Positive 😊" if pred == 1 else "Negative 😞"

demo = gr.Interface(
    fn=classify_review,
    inputs="textbox",
    outputs="text",
    title="🎬 IMDB Sentiment Classifier",
    description="Fine-tuned DistilBERT model for classifying movie review sentiment."
)
demo.launch()
